# CNN
컨볼루션 네트워크로 MNIST 98% 달성하기

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from random import randint
import shutil

## MNIST자료 불러오기
28*28사이즈의 숫자 이미지를 행렬로 표현하였으며 2차원 구조를 정규화 하여 1차원 구조로 표현되있다.
라벨의 경우 one-hot방식인데 0~9까지의 숫자를 [0,0,0,0,0,1,0,0,0,0]이런 식으로 표현하였다.

In [2]:
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print("총 학습용 데이터 갯수 : ",len(mnist.train.labels))
print("총 테스트용 데이터 갯수 : ",len(mnist.test.labels))
print("총 검증용 데이터 갯수 : ",len(mnist.validation.labels))
print("개별 데이터의 행렬 길이 : ",len(mnist.train.images[0]))
print("개별 라벨의 행렬 길이 : ", len(mnist.train.labels[0]))

총 학습용 데이터 갯수 :  55000
총 테스트용 데이터 갯수 :  10000
총 검증용 데이터 갯수 :  5000
개별 데이터의 행렬 길이 :  784
개별 라벨의 행렬 길이 :  10


## 가중치 초기화
0이 아닌 값으로 가중치를 초기해 줌으로써 원활한 학습을 할수 있게 보조한다.

In [4]:
def xaver_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0/ (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)

    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

## 파라미터 설정

In [5]:
x = tf.placeholder(tf.float32,[None,784], name="images")
label = tf.placeholder(tf.float32,[None,10],name="labels")

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))      # 3x3x1 conv, 32 outputs
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))     # 3x3x32 conv, 64 outputs
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))    # 3x3x32 conv, 128 outputs
W4 = tf.Variable(tf.random_normal([2048, 625], stddev=0.01))        # FC 128 * 4 * 4 inputs, 625 outputs
W5 = tf.Variable(tf.random_normal([625, 10], stddev=0.01))          # FC 625 inputs, 10 outputs (labels)

## 학습설정 파라미터

In [6]:
learning_rate = 0.001
# 드롭아웃
dropout_cnn_rate = tf.placeholder(tf.float32)
dropout_fcc_rate = tf.placeholder(tf.float32)
# 전체 학습 횟수
training_epochs = 10
# 단일 학습에 사용할 자료량
batch_size = 200
# 학습 결과 노출 간격
display_step = 1
# 전체 학습 자료를 단일 반복당 학습 자료수로 나누어 한 학습당 반복횟수 산정
total_batch = int(mnist.train.num_examples/batch_size)
# 텐서보드용 로그 저장 경로
log_path = "./logs/mnist4cnn_1"

## 수치 요약

In [7]:
W1_hist = tf.histogram_summary("Weights1",W1)
W2_hist = tf.histogram_summary("Weights2",W2)
W3_hist = tf.histogram_summary("Weights3",W3)
W4_hist = tf.histogram_summary("Weights4",W4)
W5_hist = tf.histogram_summary("Weights5",W5)

label_hist = tf.histogram_summary("labels",label)

## 모델 설계

In [8]:
X_image = tf.reshape(x, [-1, 28, 28, 1], name = 'X-input-reshape')

with tf.name_scope("L1") as scope:
    L1_H = tf.nn.relu(tf.nn.conv2d(X_image,W1,strides=[1,1,1,1],padding='SAME'))
    L1_P = tf.nn.max_pool(L1_H,ksize = [1,2,2,1],strides = [1,2,2,1],padding='SAME')
    L1 = tf.nn.dropout(L1_P, dropout_cnn_rate)
with tf.name_scope("L2") as scope:
    L2_H = tf.nn.relu(tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME'))
    L2_P = tf.nn.max_pool(L2_H,ksize = [1,2,2,1],strides = [1,2,2,1],padding='SAME')
    L2 = tf.nn.dropout(L2_P, dropout_cnn_rate)
with tf.name_scope("L3") as scope:
    L3_H = tf.nn.relu(tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME'))
    L3_P = tf.nn.max_pool(L3_H,ksize = [1,2,2,1],strides = [1,2,2,1],padding='SAME')
    L3_s = tf.reshape(L3_P,[-1,W4.get_shape().as_list()[0]])
    L3 = tf.nn.dropout(L3_s, dropout_cnn_rate)
with tf.name_scope("L4") as scope:
    L4_H = tf.nn.relu(tf.matmul(L3,W4))
    L4 = tf.nn.dropout(L4_H, dropout_cnn_rate)
with tf.name_scope("L-hypothesis") as scope:
    hypothesis  = tf.matmul(L4,W5)

## 비용함수(교차엔트로피)

In [9]:
with tf.name_scope("cost_func") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis,label))
    cost_summary = tf.scalar_summary("cost",cost)

with tf.name_scope("train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## 모델 평가

In [10]:
with tf.name_scope("test") as scope:
    real_label = tf.arg_max(label,1) # 실제 라벨
    learn_label = tf.arg_max(hypothesis,1) # 머신러닝 결과
    correct_prodiction = tf.equal(real_label,learn_label) # 둘다 값이 같으면 true, 틀리면 false

    # 정확도
    accuracy = tf.reduce_mean(tf.cast(correct_prodiction,tf.float32))
    accuracy_summary = tf.scalar_summary("accuracy",accuracy)

## 변수 초기화 

In [11]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


## 세션

In [12]:
sess = tf.Session()
sess.run(init)

## 텐서보드 설정

In [ ]:
merged = tf.merge_all_summaries()
try: # 이전 텐서보드 학습로그 삭제
    shutil.rmtree(log_path)
except:
    pass
writer = tf.train.SummaryWriter(log_path,sess.graph)
print("터미널에서 프로젝트가 있는 폴더에서 아래 명령어를 실행 시키면 텐서보드가 켜집니다.")
print("tensorboard --logdir={}".format(log_path))

터미널에서 프로젝트가 있는 폴더에서 아래 명령어를 실행 시키면 텐서보드가 켜집니다.
tensorboard --logdir=./logs/mnist4cnn_1


## 학습하기

In [ ]:
try:
    train_count = 0
    for epoch in range(training_epochs):
        avg_cost=0
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            _,c,cs = sess.run([optimizer,cost,cost_summary],feed_dict={x:batch_xs,label:batch_ys,dropout_cnn_rate: 0.7, dropout_fcc_rate: 0.5})
            avg_cost = c/total_batch
            writer.add_summary(cs,train_count) # 매 batch_size만큼 학습 할때마다 cost를 텐서보드에 그림
        print("{}번 학습 cost: {}".format(epoch+1,avg_cost))
        if (epoch+1) % display_step == 0:
            feed = {x:mnist.test.images, label: mnist.test.labels,dropout_cnn_rate: 1, dropout_fcc_rate: 1}
            summary,_ = sess.run([merged,accuracy],feed_dict=feed)
            writer.add_summary(summary,epoch)
except Exception as e:
    print(e)
    sess.close()
    exit()
print("학습완료")

## 테스트 데이터로 검증

In [ ]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, label:mnist.test.labels,dropout_cnn_rate: 1, dropout_fcc_rate: 1}))
sess.close()